In [1]:
import os
import tensorflow as tf
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, Activation, BatchNormalization, Conv2D, SeparableConv2D, concatenate, MaxPooling2D, GlobalAveragePooling2D 
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import matplotlib.pyplot as plt
import h5py

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\Xyrion\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
keys = np.unique(y_test)
keys

In [ ]:
values = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
classes = dict(zip(keys,values))
classes

In [ ]:
nb = np.random.choice(50000)
plt.imshow(x_train[nb], interpolation='nearest')
plt.show()
classes[y_train[nb][0]]

In [ ]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
def ddb_b(X_input, growth, repeat=6, **kwargs):
    X = X_input
    for i in range(repeat):
        X = Conv2D(growth, (1,1), strides = 1, **kwargs)(X_input)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
        X = SeparableConv2D(growth, (3,3), strides=1, **kwargs)(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
        
        X_input = concatenate([X_input, X], axis = -1)
    return X_input

In [ ]:
def tinyDSOD_backbone(input_shape, l2):
    regul = regularizers.l2(l2)
    kwargs = {'padding':'same', 'kernel_regularizer':regul,'kernel_initializer':'glorot_uniform'}

    ### STEM ###
    # Convolution 1
    inp = Input(input_shape)
    X = Conv2D(64, (3,3), strides=2, **kwargs)(inp)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Convolution 2
    X = Conv2D(64, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Depth-wise seperable convolution 1
    X = SeparableConv2D(64, (3,3), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Convolution 3
    X = Conv2D(128, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Depth-wise seperable convolution 2
    X = SeparableConv2D(128, (3,3), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Pooling
    X = MaxPooling2D((2,2), strides=2, padding='same')(X)
    
    ### Extractor ###
    # Dense stage 0
    X = ddb_b(X, 32, repeat=4, **kwargs)
    
    # Transition layer 0
    X = Conv2D(128, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2), strides=2, padding='same')(X)
    
    # Dense stage 1
    X = ddb_b(X, 48, repeat = 6, **kwargs)
    
    # Transition layer 1
    X = Conv2D(128, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2,2), strides=2, padding='same')(X)
    
    # Dense stage 2
    X = ddb_b(X, 64, repeat=6, **kwargs)
    
    # Transition layer 2
    X = Conv2D(256, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    # Dense stage 3
    X = ddb_b(X, 80, repeat=6, **kwargs)
    
    # Transition layer 3
    X = Conv2D(64, (1,1), strides=1, **kwargs)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    
    ### Classification layers ###
    X = GlobalAveragePooling2D()(X)
    X = Dense(10, activation='softmax', kernel_regularizer=regul, kernel_initializer='glorot_uniform')(X)
    
    ### Create Model ### 
    
    model = Model(inputs=inp, outputs=X, name='TinyDSOD_bb')
    
    return model

In [ ]:
classification_model = tinyDSOD_backbone((32,32,3), l2=5e-4)

In [ ]:
classification_model.summary()

In [ ]:
datagen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',
        horizontal_flip=True)

In [ ]:
for num in range(5):
    model_name = 'MODEL-%01d.h5' % num
    print(model_name)
    
     #We create a checkpoint to save the best model and add an early stopping
    checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='min')
    lr_annealing = ReduceLROnPlateau(monitor='val_loss', patience = 2, epsilon=0.001, factor = 0.25, min_lr = 1e-8, verbose = 1, mode='min' )
    callbacks_list = [checkpoint, early_stop, lr_annealing]
    
    from keras.optimizers import SGD

    sgd = SGD(lr=0.1, momentum=0.9)
    classification_model.compile(optimizer = sgd, loss = "categorical_crossentropy", metrics=['accuracy'])

    classification_model.fit_generator(
                            datagen.flow(x_train, y_train, batch_size=32),
                            epochs=200,
                            validation_data=(x_test, y_test),
                            workers=4,
                            shuffle = True,
                            callbacks=callbacks_list)